In [2]:
import torch
import torch.nn.functional as F

class GCNLayer(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.linear = torch.nn.Linear(in_features, out_features)

    def forward(self, x, adjacency):
        x = self.linear(x)
        x = torch.matmul(adjacency, x)
        return F.relu(x)

# 假设输入特征维度为 10，输出特征维度为 5
input_features = 5
output_features = 2


adjacency = torch.randn(5, 5)

# 创建 GCN 层
gcn_layer1 = GCNLayer(input_features, output_features)
gcn_layer2 = GCNLayer(output_features, output_features)
gcn_layer3 = GCNLayer(output_features, output_features)

# 输入特征
x = torch.randn(5, input_features)

# 执行三层 GCN 卷积
x1 = gcn_layer1(x, adjacency)
x2 = gcn_layer2(x1, adjacency)
x3 = gcn_layer3(x2, adjacency)
print(x.shape, '\n', x)
print("Layer 1 output:")
print(x1.shape, '\n', x1)
print("Layer 2 output:")
print(x2.shape,'\n', x2)
print("Layer 3 output:")
print(x3.shape,'\n', x3)


torch.Size([5, 5]) 
 tensor([[-1.3473, -1.4131, -0.8441, -0.1647,  0.4423],
        [ 0.0794,  1.4567, -0.9358,  0.1678,  2.5868],
        [-0.5945, -0.3043,  0.5799,  1.2134, -1.4947],
        [ 1.0555, -0.6432, -1.2222,  0.5167,  0.8901],
        [-1.8409,  0.5959,  1.0114, -1.4287,  0.1060]])
Layer 1 output:
torch.Size([5, 2]) 
 tensor([[0.6964, 1.1877],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.3420, 0.9330],
        [0.0000, 0.0000]], grad_fn=<ReluBackward0>)
Layer 2 output:
torch.Size([5, 2]) 
 tensor([[0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.5878, 0.0000],
        [0.1381, 0.0000],
        [0.0000, 0.4541]], grad_fn=<ReluBackward0>)
Layer 3 output:
torch.Size([5, 2]) 
 tensor([[0.1219, 0.0000],
        [0.0000, 0.7525],
        [0.0000, 1.5406],
        [0.0000, 1.0165],
        [1.4883, 0.0000]], grad_fn=<ReluBackward0>)


In [7]:
import os
import numpy as np
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from scipy.sparse import csr_matrix
import networkx as nx

from scipy.linalg import orthogonal_procrustes

import torch
import torch.optim as optim
from torch_geometric.data import Data

from sklearn.manifold import TSNE
from sklearn.datasets import make_moons  # 导入make_moons函数

from util import Net, GIN, GAT, stationary, reconstruct, dG


np.random.seed(0)
torch.manual_seed(0)

n = 2000
m = 500
DISTANCE = 0.1

# 导入moon数据集
x, y = make_moons(n_samples=n, noise=0.1, random_state=0) 
# 分割训练集和测试机
n_train = int(n * 0.7)
train_ind = torch.randperm(n)[:n_train]
test_ind = torch.LongTensor(list(set(np.arange(n)) - set(train_ind.tolist())))
# K = int(np.sqrt(n) * np.log2(n) / 10) #作为隐藏输入
D = pairwise_distances(x)# 邻接距离矩阵
# 二值化，得到邻接矩阵
A_binary = np.where(D <= DISTANCE, 1, 0)
# A_binary
# 获得所有的边，二元组格式
row_indices, col_indices = np.where(A_binary == 1)
edges = list(zip(row_indices, col_indices))

# 转为tensor
x = torch.tensor(x, dtype=torch.float)
y = torch.tensor(y, dtype=torch.long)
net = GIN(m)
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Train the GIN network
for epoch in range(10):
    ind = torch.eye(n)[:, torch.randperm(n)[:m]]
    X_extended = torch.hstack([torch.tensor(x), ind])
    rec = net(X_extended, torch.tensor(A_binary))
    loss = dG(x[train_ind], rec[train_ind])
    print(f'Epoch {epoch + 1}, Loss: {float(loss)}')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Calculate node embeddings and reconstruction loss
rec_GIN = rec.detach().numpy() @ R.T
loss_GIN = float(dG(x, rec))

# Create figure
fig, ax = plt.subplots(1, 2, figsize=(14, 4))

# Plot ground truth
ax[0].scatter(x[:, 0], x[:, 1], c=y, s=10, cmap='viridis', rasterized=True)
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].set_facecolor('#eeeeee')
txt = ax[0].text(0.05, 0.05, 'Ground Truth', color='k', fontsize=14, weight='bold', transform=ax[0].transAxes)
txt.set_path_effects([PathEffects.withStroke(linewidth=5, foreground='#eeeeee')])

# Plot reconstructed embeddings
ax[1].scatter(rec_GIN[:, 0], rec_GIN[:, 1], c=y, s=10, cmap='viridis', rasterized=True)
ax[1].set_xticks([])
ax[1].set_yticks([])
txt = ax[1].text(0.05, 0.05, f'GIN $d_G = {loss_GIN:.2f}$', color='k', fontsize=14, weight='bold',
                 transform=ax[1].transAxes)
txt.set_path_effects([PathEffects.withStroke(linewidth=5, foreground='w')])

fig.subplots_adjust()

if not os.path.exists('imgs'):
    os.mkdir('imgs')

# fig.savefig('imgs/semi_moon_GIN.png', bbox_inches='tight', dpi=300)

plt.show()

<ipython-input-7-6a4614ce3e61>:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_extended = torch.hstack([torch.tensor(x), ind])


TypeError: forward() takes 2 positional arguments but 3 were given